In [3]:
##### DOWNLOAD DATASET #####
from IPython.display import clear_output
from google.colab import files
files.upload()
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d gsimonx37/letterboxd
!unzip /content/letterboxd.zip -d /content/letterbox/
clear_output()
############################

In [1]:
########### SPARK CONTEXT #####################
import pandas as pd
import itertools
from tqdm import tqdm

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better

sc = spark.sparkContext
###############################################

In [50]:
############## BASKET CREATION ################
#TODO do it using SPARK directly -> cars = spark.read.csv('cars.csv', header=True, sep=";")
actors = pd.read_csv("letterbox/actors.csv")
actors = actors
baskets = actors.groupby("id")["name"].apply(list)
#baskets = baskets.sample(10000)

print("number of baskets: " + str(len(baskets)))
print("biggest basket: " + str(baskets.map(len).max()))
print(baskets)
baskets_RDD = sc.parallelize(baskets).cache()
##############################################

number of baskets: 603163
biggest basket: 451
id
1000001    [Margot Robbie, Lewis Easter, Onyemachi Ejimof...
1000002    [Rosie Peralta, Anna Elisabeth Rihlmann, Andre...
1000003    [Randall Archer, Boon Pin Koh, Efka Kvaracieju...
1000004    [Leonard Termo, Greg Bronson, Michael Arturo, ...
1000005    [Lena Georgas, Jeff Hephner, Elyes Gabel, Broo...
                                 ...                        
1896377    [Marat Basharov, Sergey Rost, Gleb Kalyuzhny, ...
1896382                                   [Marine Petrosyan]
1896387                                        [Rebecca Jim]
1896391                                       [Ko Shibasaki]
1896393                                    [Atsuhiro Inukai]
Name: name, Length: 603163, dtype: object


In [52]:
test_basket = [
    ['a','b','c','d','e','f','g','h','i','l'],
    ['n','o'],
    ['o','n'],
    ['a','b','c','d','e','f',],
    ['a','b','c'],
    ['f','q'],
    ['f','q','z'],
    ['f','u','q'],
    ['n','o','i'],
    ['a','b','c','d','e','f','q','p','o','n'],
    ['a','b','c','d','e','f','r','s','t','u'],
]
test_RDD = sc.parallelize(test_basket).cache()

In [33]:
## APRIORI
def apriori(chunk, s, tot_bsk):

  n_bsk = 0 # number of baskets in chunk used to calculate support
#  trsl = dict() # using a translation could be better for the combination step
  count = dict() # should be relatively efficient since is a hash table

#
#  def translate(e, dic):
#    try:
#      return dic[e]
#    except KeyError:
#      dic[e] = len(dic)+1
#      return dic[e]
  def incr(e,dic):
    try:
      dic[e] += 1;
    except KeyError:
      dic[e] = 1

  chunk = list(chunk) # to avoid consuming after first pass
  ### first pass
  for bsk in chunk:
    n_bsk += 1;
    #elements = [translate(x,trsl) for x in bsk]
    for b in bsk:
      incr(b,count)

  ### filter
  to_del = []
  frequent = []
  for c in count:
    if count[c] >= s* n_bsk/tot_bsk:
      frequent.append(c)
      #yield (c,1)

  """

  #frequent = list(count.keys())
  #yield (len(frequent),1)

  frequent.sort()
  cmb = list(itertools.combinations(frequent,2)) #It's probably more efficient to
  #yield (len(cmb),1)

  count_2 = dict()
  ### second pass
  for bsk in chunk:
    for pair in cmb: #this already solve the ordering problem if the list is ordered
      if pair[0] in bsk and pair[1] in bsk:
        incr(tuple(pair), count_2)

  to_del = []
  frequent = []
  for c in count_2:
    if count_2[c] < (s * n_bsk)/tot_bsk:
      to_del.append(c)
    else:
      frequent.append(c)
      yield(c,1)
  for t in to_del:
    del count_2[t]
  """
  k = 2
  while True:#len(frequent) > 0:
    print(frequent)
    if len(frequent) == 0:
      break
    frequent.sort()
    #cmb = list(itertools.combinations(frequent, 3))
    count = dict()

    for bsk in tqdm(chunk):
      #print('basket')
      #bsk = [x for x in bsk if x in frequent] #filter only members of Ck
      filtered = set()
      for b in bsk:
        for f in frequent:
          if b in f:
            filtered.add(b)
            break;
      bsk = list(filtered)
      #print('bsk:'+str(bsk))
      bsk.sort()
      for tpl in itertools.combinations(bsk, k):
          incr(tuple(tpl), count)

    frequent = []
    for c in count:
      if count[c] >= s * n_bsk/tot_bsk:
        frequent.append(c)
        yield(c,1)
    #print(len(frequent))
    k+=1






In [34]:
real = []
for a in apriori(test_basket,5,len(test_basket)):
  real.append(a)
print(real)

['a', 'b', 'c', 'f']


100%|██████████| 11/11 [00:00<00:00, 45907.80it/s]


[('a', 'b'), ('a', 'c'), ('b', 'c')]


100%|██████████| 11/11 [00:00<00:00, 51093.40it/s]


[('a', 'b', 'c')]


100%|██████████| 11/11 [00:00<00:00, 61028.23it/s]

[]
[(('a', 'b'), 1), (('a', 'c'), 1), (('b', 'c'), 1), (('a', 'b', 'c'), 1)]


In [80]:
S =  5; #support threshold (maybe a fraction is better)
TOT = len(test_basket)#len(baskets)

#baskets_RDD
#SON first step
candidates = test_RDD.mapPartitions(lambda x: apriori(x,S,TOT)).reduceByKey(lambda a,b: a).keys().collect()
print(candidates)

[('b', 'c'), ('a', 'b', 'c'), ('f', 'q'), ('a', 'b'), ('a', 'c')]


In [70]:
not_real = [x for x in candidates if x not in real]
not_present = [x for x in real if x not in candidates]
print(not_real)
print(not_present)

[(('f', 'q'), 1)]
[]


In [79]:
test_candidates = [('b','c'), ('a','b','c'), ('f','q'), ('a','b'), ('a','c')]
#SON second step
def second_map(chunk,itemset,s):
  def incr(e,dic):
    try:
      dic[e] += 1;
    except KeyError:
      dic[e] = 1

  count = dict()
  for bsk in tqdm(chunk):
    for i in itemset:
      flag = True
      for e in i:
        if e not in bsk:
          flag = False
          break
      if flag:
        incr(i,count)
  for c in count:
    yield (c,count[c])

real_from_candidates = test_RDD.mapPartitions(lambda x: second_map(x, test_candidates,S)).reduceByKey(lambda a,b: a+b).filter(lambda a: a[1] >= S).keys().collect()
print(real_from_candidates)
for r in real_from_candidates:
  print(r)
#for i in second_map(test_basket, test_candidates,s):
#  print(i)

[('b', 'c'), ('a', 'b', 'c'), ('a', 'b'), ('a', 'c')]
('b', 'c')
('a', 'b', 'c')
('a', 'b')
('a', 'c')


In [67]:
type(s)

tuple